In [1]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib notebook

from lowe.locations.lookup import name2fips, fips2name, search, generate_df_json
from lowe.acs.ACSClient import ACSClient

In [2]:
cities = ['cathedral city', 'coachella', 'desert hot springs', 'indian wells', 'indio', 'la quinta', 'palm desert', 'palm springs', 'rancho mirage']

In [3]:
search(query = "desert hot springs", codetype = "city", search_on = "name")

                        name     fips
2400  desert hot springs, ca  0618996


--

In [4]:
loc = {
    "state": "06",
    "city": "18996"
}

In [5]:
namedict = fips2name(loc)
name2fips(namedict)

{'city': '0618996', 'state': '06'}

In [6]:
temp_client = ACSClient()
await temp_client.initialize()

In [12]:
temp_resp = await temp_client.get_acs(vars = ["S1901"],
                     start_year="2019",
                     end_year="2019",
                     location=loc,
                     estimate="5")

In [14]:
temp_resp["INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Households Total"]

year
2019    10476
Name: INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Households Total, dtype: object

In [9]:
for col in list(temp_resp.columns):
    print(col)

INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Families Mean income (dollars)
INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Families Median income (dollars)
INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Families PERCENT ALLOCATED Family income in the past 12 months
INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Families PERCENT ALLOCATED Household income in the past 12 months
INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Families PERCENT ALLOCATED Nonfamily income in the past 12 months
INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Families Total
INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Families Total $10,000 to $14,999
INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Families Total $100,000 to $149,999
INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED

In [10]:
temp_resp = await temp_client.get_acs(vars = ["DP05"],
                     start_year="2019",
                     end_year="2019",
                     location=loc,
                     estimate="5")

In [11]:
for col in list(temp_resp.columns):
    print(col)

ACS DEMOGRAPHIC AND HOUSING ESTIMATES Estimate CITIZEN, VOTING AGE POPULATION Citizen, 18 and over population Female
ACS DEMOGRAPHIC AND HOUSING ESTIMATES Estimate CITIZEN, VOTING AGE POPULATION Citizen, 18 and over population Male
ACS DEMOGRAPHIC AND HOUSING ESTIMATES Estimate HISPANIC OR LATINO AND RACE Total population
ACS DEMOGRAPHIC AND HOUSING ESTIMATES Estimate HISPANIC OR LATINO AND RACE Total population Hispanic or Latino (of any race)
ACS DEMOGRAPHIC AND HOUSING ESTIMATES Estimate HISPANIC OR LATINO AND RACE Total population Hispanic or Latino (of any race) Cuban
ACS DEMOGRAPHIC AND HOUSING ESTIMATES Estimate HISPANIC OR LATINO AND RACE Total population Hispanic or Latino (of any race) Mexican
ACS DEMOGRAPHIC AND HOUSING ESTIMATES Estimate HISPANIC OR LATINO AND RACE Total population Hispanic or Latino (of any race) Other Hispanic or Latino
ACS DEMOGRAPHIC AND HOUSING ESTIMATES Estimate HISPANIC OR LATINO AND RACE Total population Hispanic or Latino (of any race) Puerto Rican

---

# Past

async def cityReport_fig5(cities: list = ['desert hot springs, ca', 'coachella, ca'], save: bool = False, save_path: str = None):
    """
    Parameters
    ----------
    city: name of the city eg. 
    target
    save: bool
    True or False, whether or not you want to save
    save_path: str
    Path to save the file to
    """
    target_cols = "TYPES OF COMPUTERS AND INTERNET SUBSCRIPTIONS Estimate Percent Total households TYPE OF INTERNET SUBSCRIPTIONS With an Internet subscription: Broadband of any type"
    new_col_name = "Percent of Total Households with Broadband"
    
    
    cols = {
        # old col name: new name
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent SEX AND AGE Total population Under 5 years": "Under 5",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent SEX AND AGE Total population 5 to 9 years": "5 to 9",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent SEX AND AGE Total population 10 to 14 years": "10 to 14",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent SEX AND AGE Total population 15 to 19 years": "15 to 19",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent SEX AND AGE Total population 20 to 24 years": "20 to 24",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent SEX AND AGE Total population 25 to 34 years": "25 to 34",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent SEX AND AGE Total population 35 to 44 years": "35 to 44",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent SEX AND AGE Total population 45 to 54 years": "45 to 54",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent SEX AND AGE Total population 55 to 59 years": "55 to 59",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent SEX AND AGE Total population 60 to 64 years": "60 to 64",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent SEX AND AGE Total population 65 years and over": "65 over"
    }
    
    
    loc_fips = city_search(cities)
    
    resp = await acs_data(loc_fips, "2019", "DP05")
    
    col_sub = [*list(cols.keys()), "state", "city"]
    resp = resp[col_sub]
    resp = resp.rename(columns = cols)
    
    for col in cols.values():
        resp[col] = pd.to_numeric(resp[col])
    
    resp["0 to 14"] = resp["Under 5"] + resp["5 to 9"] + resp["10 to 14"]
    resp["15 to 24"] = resp["15 to 19"] + resp["20 to 24"]
    resp["45 to 65"] = resp["45 to 54"] + resp["55 to 59"] + resp["60 to 64"]
    
    return resp
    
    
    # Following is unique to each section
    
    categ = resp.columns[0:8]
    value = resp.iloc[0][0:8].astype(str).astype(float)
    

    plot_df = pd.DataFrame({'Race': categ, 'Percentage': value})
    
    fig = px.bar(plot_df, x="Race", y="Percentage", text='Percentage')
    fig.update_xaxes(type='category')
    
    if save:
        fig.write_image(save_path)
    return fig
    